In [ ]:
import cv2
import os
import ffmpeg

def extract_frames(video_path, output_dir, video_dict, frame_dict):
    """Extracts frames from a video and saves them to a directory."""

    # Converted video path
    converted_video_path = os.path.join(output_dir, "converted_video.mp4")

    # Skip if the converted video already exists
    if os.path.exists(converted_video_path):
        print(f"Converted video already exists: {converted_video_path}. Skipping conversion.")
    else:
        try:
            ffmpeg.input(video_path).output(converted_video_path, vcodec="libx264", acodec="aac").run(cmd="/opt/homebrew/bin/ffmpeg")
            print(f"Converted {video_path} to {converted_video_path}")
        except Exception as e:
            print(f"Error converting {video_path}: {e}")
            return

    """Saves the frames specified in frame_dict as images."""
    # Get the video name (without extension)
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Get the total number of frames for this video from video_dict
    NoOfFrames = video_dict.get(video_name, 0)

    # Get the list of frames to extract from frame_dict, ensuring they are within the valid range
    valid_frames = [frame for frame in frame_dict.get(video_name, []) if frame < NoOfFrames]
    print(f"Valid frames for {video_name}: {valid_frames}")
    
    # Open the converted video file
    cap = cv2.VideoCapture(converted_video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {converted_video_path}")
        return

    cap.release()  # Release the video file
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {converted_video_path}")



In [6]:
import pandas as pd

# Load the data
file_path = '/Users/vrishfish/DS Project/EchoNet-Dynamic/FileList.csv'  
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())

# Select only the two columns: 'VideoName' and 'NumSplits' (replace with your actual column names)
selected_columns = data[['FileName', 'NumberOfFrames']]

# Convert to a dictionary
video_dict = selected_columns.set_index('FileName')['NumberOfFrames'].to_dict()

# Display the dictionary
print(video_dict)



             FileName         EF         ESV         EDV  FrameHeight  \
0  0X100009310A3BD7FC  78.498406   14.881368   69.210534          112   
1  0X1002E8FBACD08477  59.101988   40.383876   98.742884          112   
2  0X1005D03EED19C65B  62.363798   14.267784   37.909734          112   
3  0X10075961BC11C88E  54.545097   33.143084   72.914210          112   
4  0X10094BA0A028EAC3  24.887742  127.581945  169.855024          112   

   FrameWidth  FPS  NumberOfFrames  Split  
0         112   50             174    VAL  
1         112   50             215  TRAIN  
2         112   50             104  TRAIN  
3         112   55             122  TRAIN  
4         112   52             207    VAL  
{'0X100009310A3BD7FC': 174, '0X1002E8FBACD08477': 215, '0X1005D03EED19C65B': 104, '0X10075961BC11C88E': 122, '0X10094BA0A028EAC3': 207, '0X100CF05D141FF143': 248, '0X100E3B8D3280BEC5': 101, '0X100E491B3CD58DE2': 226, '0X100F044876B98F90': 221, '0X101026B90DAE7E95': 147, '0X1012703CDC1436FE': 201,

In [10]:
import pandas as pd

# Load the data
file_path = '/Users/vrishfish/DS Project/EchoNet-Dynamic/VolumeTracings.csv'  
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())

# Select only the two columns: 'VideoName' and 'NumSplits' (replace with your actual column names)
selected_columns = data[['FileName', 'Frame']]

# Group by 'FileName' and use a set to avoid duplicate frames
frame_dict = (
    selected_columns.groupby('FileName')['Frame']
    .apply(lambda x: list(set(x)))  # Remove duplicate frames using a set
    .to_dict()
)

# Display the dictionary
print(frame_dict)

                 FileName         X1         Y1         X2         Y2  Frame
0  0X100009310A3BD7FC.avi  51.260417  15.348958  64.932292  69.125000     46
1  0X100009310A3BD7FC.avi  50.037611  17.167841  53.367222  16.321330     46
2  0X100009310A3BD7FC.avi  49.157378  20.407629  57.090549  18.390722     46
3  0X100009310A3BD7FC.avi  48.538173  23.581055  59.997339  20.667707     46
4  0X100009310A3BD7FC.avi  47.918968  26.754480  62.904129  22.944693     46
{'0X100009310A3BD7FC.avi': [61, 46], '0X1002E8FBACD08477.avi': [18, 3], '0X1005D03EED19C65B.avi': [24, 35], '0X10075961BC11C88E.avi': [91, 108], '0X10094BA0A028EAC3.avi': [137, 156], '0X100CF05D141FF143.avi': [148, 132], '0X100E3B8D3280BEC5.avi': [25, 38], '0X100E491B3CD58DE2.avi': [49, 75], '0X100F044876B98F90.avi': [56, 72], '0X101026B90DAE7E95.avi': [45, 62], '0X1012703CDC1436FE.avi': [154, 172], '0X1013E8A4864781B.avi': [35, 46], '0X1018521A3BC5CDBA.avi': [57, 42], '0X101C388397F66EDB.avi': [62, 46], '0X101CFC9C5351DCBE.avi': [7

In [ ]:
# Define input and output directories
input_dir = "/Users/vrishfish/DS Project/EchoNet-Dynamic/Videos"
output_dir = "/Users/vrishfish/DS Project/Video_frames"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each video in the input directory
for file_name in os.listdir(input_dir):

    if file_name.startswith("."):  # Skip hidden files
        continue

    video_path = os.path.join(input_dir, file_name)  # Construct the full path to the video

    if not file_name.endswith((".avi", ".mp4", ".mov")):  # Skip non-video files
        print(f"Skipping non-video file: {file_name}")
        continue

    print(f"Processing: {file_name}")

    # Create a subdirectory for frames from this video
    video_frame_dir = os.path.join(output_dir, os.path.splitext(file_name)[0])
    os.makedirs(video_frame_dir, exist_ok=True)

    # Extract frames
    extract_frames(video_path, video_frame_dir, video_dict, frame_dict)